# PRIVACY POLICY ANALYSER BOT 

### step 1 :

In [7]:
# Cell 1: Install dependencies (Updated)
!pip install langchain langchain-community langchain-openai langchain-text-splitters langgraph pydantic pypdf python-dotenv tiktoken faiss-cpu langsmith tavily-python gradio

In [8]:
# Cell 1: Install compatible LangChain version (v0.1.20)
!pip install --force-reinstall langchain==0.1.20 langchain-community==0.0.38 langchain-core==0.1.52 langchain-openai==0.1.6 langgraph pydantic pypdf python-dotenv tiktoken faiss-cpu langsmith tavily-python gradio --force-reinstall

  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.38-py3-none-any.whl.metadata (8.7 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_openai-0.1.6-py3-none-any.whl.metadata (2.5 kB)
  Using cached langgraph-1.0.4-py3-none-any.whl.metadata (7.8 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached pypdf-6.4.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached tiktoken-0.12.0-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached faiss_cpu-1.13.0-cp39-abi3-macosx_14_0_arm64.whl.metadata (7.7 kB)
  Using cached langsmith-0.4.49-py3-none-any.whl.metadata (14 kB)
  Using cached tavily_python-0.7.13-py3-none-any.whl.metadata (7.5 kB)
  Using cached gradio-6.0.1-py3-none-any.whl.metadata (16 kB)
  Using cached pyyaml-6.0.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Usin

In [9]:
# Cell 1: Install dependencies from requirements.txt
import os


# Install
!pip install -r requirements.txt

print("All dependencies installed successfully!")

All dependencies installed successfully!


In [10]:
pip install langchain.text_splitters

Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [12]:
# Cell 2: Import libraries and setup
import os
import json
import tempfile
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv
from pypdf import PdfReader

# LangChain imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

# LangGraph imports
from langgraph.graph import StateGraph, END

# LangSmith imports for tracing
from langsmith import Client
from langchain.callbacks.tracers import LangChainTracer

# Pydantic for structured output
from pydantic import BaseModel, Field

# ipywidgets for UI
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Image, display

# Load environment variables (ensure .env file has OPENAI_API_KEY and TAVILY_API_KEY)
load_dotenv()

# Set mock API keys if not present (replace with actual keys for production)
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY", "YOUR_OPENAI_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY", "YOUR_TAVILY_KEY")

print("Environment setup complete!")
print(" All necessary libraries imported.")

Environment setup complete!
 All necessary libraries imported.
